In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [82]:
#PREPROCESS DATA
df = pd.read_csv('clean_data.csv')
df = df.drop(['Unnamed: 0', 'BS_trend', 'date', 'hour', 'minute'], axis='columns')

df = pd.get_dummies(df, columns=['weekday'])

In [83]:

X = df.drop('BS', axis='columns')
y = df['BS']

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [91]:
X_train.shape

(4378, 19)

In [92]:
model = tf.keras.Sequential([
    layers.Dense(1, activation='relu')
])

In [93]:
model.compile(loss='mean_absolute_error', optimizer='adam')
model(X_train.to_numpy())
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (4378, 1)                 20        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [94]:
history = model.fit(x=X_train,y=y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
110/110 [==============================] - 0s 2ms/step - loss: 110.8174 - val_loss: 86.3053
Epoch 2/100
110/110 [==============================] - 0s 903us/step - loss: 103.1605 - val_loss: 78.0922
Epoch 3/100
110/110 [==============================] - 0s 850us/step - loss: 95.4566 - val_loss: 69.8962
Epoch 4/100
110/110 [==============================] - 0s 891us/step - loss: 87.7954 - val_loss: 62.0315
Epoch 5/100
110/110 [==============================] - 0s 887us/step - loss: 80.2398 - val_loss: 54.9572
Epoch 6/100
110/110 [==============================] - 0s 856us/step - loss: 73.0591 - val_loss: 48.6032
Epoch 7/100
110/110 [==============================] - 0s 916us/step - loss: 66.4417 - val_loss: 43.1029
Epoch 8/100
110/110 [==============================] - 0s 978us/step - loss: 60.7643 - val_loss: 38.8434
Epoch 9/100
110/110 [==============================] - 0s 899us/step - loss: 56.2724 - val_loss: 35.5773
Epoch 10/100
110/110 [==============================] -

110/110 [==============================] - 0s 810us/step - loss: 42.9652 - val_loss: 35.1322
Epoch 79/100
110/110 [==============================] - 0s 779us/step - loss: 42.9618 - val_loss: 35.2188
Epoch 80/100
110/110 [==============================] - 0s 948us/step - loss: 42.9350 - val_loss: 35.1158
Epoch 81/100
110/110 [==============================] - 0s 871us/step - loss: 42.9236 - val_loss: 34.9790
Epoch 82/100
110/110 [==============================] - 0s 841us/step - loss: 42.8999 - val_loss: 35.0283
Epoch 83/100
110/110 [==============================] - 0s 830us/step - loss: 42.8788 - val_loss: 35.0878
Epoch 84/100
110/110 [==============================] - 0s 850us/step - loss: 42.8616 - val_loss: 35.0032
Epoch 85/100
110/110 [==============================] - 0s 893us/step - loss: 42.8444 - val_loss: 34.9932
Epoch 86/100
110/110 [==============================] - 0s 893us/step - loss: 42.8361 - val_loss: 35.0564
Epoch 87/100
110/110 [==============================] - 0s 

In [71]:
predicted = model.predict(X_test)
predicted[:10]

array([[132.23705],
       [130.34818],
       [131.76485],
       [131.76485],
       [132.70927],
       [133.18146],
       [133.65369],
       [132.70927],
       [132.23705],
       [132.23705]], dtype=float32)

In [72]:
y_test[:10]

4378    106.0
4379    104.0
4380    101.0
4381    102.0
4382    104.0
4383    111.0
4384    122.0
4385    131.0
4386    132.0
4387    126.0
Name: BS, dtype: float64

In [73]:
mean_squared_error(y_test, predicted)

1383.5147162499059